In [56]:
import csv
import json
from pprint import pprint
def save_json(data, filepath):
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
csv_file = open('HCI_paper_abstracts/HCI_paper_per_record_release.csv', 'r')
reader = csv.DictReader(csv_file)
keys = ['paperid', 'abstract', 'title']
extracted_data = []
for row in reader:
    # exclusion criteria:
    # 1. title and abstract are empty
    if row['title'] == '' or row['abstract'] == '':
        continue
    # 2. is not UIST paper
    if row['venue'] != 'UIST': 
        continue
    # 3. is not a publication
    if row['title'].startswith("Proceedings of the"):
        continue
    extracted_data.append({key: row[key] for key in keys})

for index, paper in enumerate(extracted_data):
    # paper['id'] = paper['paperid']
    paper['id'] = str(index)
    paper['content'] = paper['title'] + '\n' + paper['abstract']
    print(paper['content'])
    print("=============================")
    del paper['paperid']
    del paper['title']
    del paper['abstract']
save_json(extracted_data, 'HCI_paper_abstracts/papers.json')
    

Multimodal agent interface based on dynamical dialogue model: MAICO: multimodal agent interface for communication
In this paper, we describe a multimodal interface prototype system based on Dynamical Dialogue Model. This system not only integrates information of speech and gestures, but also controls the response timing in order to realize a smooth interaction between user and computer. Our approach consists of human-human dialogue analysis, and computational modeling of dialogue.
Rapid Prototyping of Multimodal Interactive Applications Based on Off-The-Shelf Heterogeneous Components
OpenInterface Kernel is a lightweight open-source plat-form designed for supporting the effective prototyping of multimodal interactive systems. Iterative design of such applications requires the easy integration, replacement, interconnection or upgrade of components. OpenInterface provides a thin integration platform able to manage these key elements with little programming knowledge, and thus provide the

In [2]:
import json
from pprint import pprint
def save_json(data, filepath):
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
paper_data = json.load(open("HCI_paper_abstracts/papers.json"))
print(len(paper_data))
paper_data_small = paper_data[:100]
save_json(paper_data_small, 'HCI_paper_abstracts/papers_small.json')

1723


In [33]:
import asyncio
import tiktoken
from openai import OpenAI
import traceback

from tqdm.asyncio import tqdm_asyncio
async def multithread_embeddings(texts, api_key):
    tasks = [get_embedding(text, api_key) for text in texts]
    # results = await asyncio.gather(*tasks)  # Runs network requests concurrently
    results = await tqdm_asyncio.gather(*tasks, desc="generating embeddings")
    return results


async def get_embedding(text, api_key, model="text-embedding-3-small"):
    return await asyncio.to_thread(_get_embedding_block, text, api_key, model)

def _get_embedding_block(text, api_key, model="text-embedding-3-small"):
    client = OpenAI(api_key=api_key)
    enc = tiktoken.encoding_for_model(model)
    # print("tokens: ", len(enc.encode(text)), len(enc.encode(text)) > 8191)
    while len(enc.encode(text)) > 8191:
        text = text[:-100]
        print("truncated: ", len(enc.encode(text)))
    try:
        return client.embeddings.create(input=text, model=model).data[0].embedding
    except Exception as e:
        print(traceback.format_exc()) 
        # return get_embedding(text, api_key, model)

# api_key = open("../api_key").read().strip()
# paper_data = json.load(open("HCI_paper_abstracts/papers.json"))
# paper_data = paper_data[:2]
# texts = [paper['content'] for paper in paper_data]
# embeddings = await multithread_embeddings(texts, api_key)


In [28]:
import random
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent

async def call_agent(agent, user_message):
    response = await agent.on_messages(
        [TextMessage(content=user_message, source="user")],
        cancellation_token=CancellationToken(),
    )
    return response


async def parallel_call_agents(agent, user_messages):
    tasks = [call_agent(agent, user_message) for user_message in user_messages]

    results = await asyncio.gather(*tasks)
    return results

def generate_topic_assignment_agent(model: str, api_key: str):
    model_client = OpenAIChatCompletionClient(
        model=model,
        api_key=api_key,
        temperature=0.0,
        model_capabilities={
            "vision": False,
            "function_calling": False,
            "json_output": True,
        },
    )
    agent = AssistantAgent(
        name="goal_decomposition_agent",
        model_client=model_client,
        system_message="""You are a topic assignment system. The user will provide you with a list of texts. You need to assign one topic to summarize all of them. 
            The topic should be a simple noun-phrase. Only one topic should be generated.
            Reply with the JSON format: 
            {{
                topic: string 
            }}""",
    )
    return agent


paper_data = json.load(open("HCI_paper_abstracts/papers.json"))
all_texts = [paper['content'] for paper in paper_data]
cluster_texts = {
    "0": all_texts[:180]
}
model="gpt-4o-mini"
api_key = open("../api_key").read().strip()
topic_assignment_agent = generate_topic_assignment_agent(model, api_key)
user_messages = []
for cluster, texts in cluster_texts.items():
    enc = tiktoken.encoding_for_model("gpt-4o-mini")
    if (
        len(enc.encode("\n".join(texts))) > 100000
    ):  # 128000 is the max token limit for GPT-4o-mini
        texts = random.sample(texts, len(texts) // 2)
    user_messages.append("\n".join(texts))

responses = await parallel_call_agents(topic_assignment_agent, user_messages)
responses = [
    json.loads(response.chat_message.content)["topic"] for response in responses
]

In [29]:
responses

['Multimodal Interaction and User Interface Design']

In [54]:
import numpy as np
from sklearn.cluster import AgglomerativeClustering, OPTICS
from sklearn.metrics.pairwise import cosine_distances
from scipy.optimize import minimize
from collections import defaultdict

def circular_dr(X):
    print("calculating pairwise distances")
    D = cosine_distances(X)
    print("optimizing positions")
    angles = optimize_positions(D)
    return angles


def optimize_positions(D, initial_theta=None, verbose=False):
    """
    Optimize the positions of points on a circle to preserve the distances in D.

    Parameters:
    - D: n x n distance matrix.
    - initial_theta: Optional initial guess for the angles.
    - verbose: If True, print optimization details.

    Returns:
    - Optimized angles theta (in radians).
    """
    n = D.shape[0]

    d_min = np.min(D)
    d_max = np.max(D)
    D = (D - d_min) / (d_max - d_min)
    if initial_theta is None:
        # Initialize theta randomly between 0 and 2pi
        initial_theta = np.random.uniform(0, 2 * np.pi, n)

    # Define bounds for each theta: [0, 2pi]
    bounds = [(0, 2 * np.pi) for _ in range(n)]

    # Optionally, fix the first angle to 0 to remove rotational symmetry
    # Uncomment the following lines if you want to fix theta_0 = 0
    # bounds = [(0, 0)] + [(0, 2 * np.pi) for _ in range(n - 1)]

    # Define the objective function with D fixed
    obj = lambda theta: objective(theta, D)

    # Perform the optimization
    result = minimize(
        obj,
        initial_theta,
        method="L-BFGS-B",
        bounds=bounds,
        options={"disp": True, "maxfun": 9999999},
    )

    if not result.success:
        raise ValueError("Optimization failed: " + result.message)

    optimized_theta = result.x
    return optimized_theta


def objective(theta, D):
    """
    Objective function to minimize: sum of squared differences between
    circular distances and the given distance matrix D.

    Parameters:
    - theta: array of angles (in radians) for each point.
    - D: n x n distance matrix.

    Returns:
    - Sum of squared differences.
    """
    n = len(theta)
    # Compute pairwise circular distances
    d = 1 - np.cos(
        np.minimum(
            np.abs(theta[:, np.newaxis] - theta[np.newaxis, :]),
            2 * np.pi - np.abs(theta[:, np.newaxis] - theta[np.newaxis, :]),
        )
    )

    # Compute the difference only for i < j to avoid double counting and zero diagonals
    mask = np.triu(np.ones((n, n)), k=1).astype(bool)
    diff = d[mask] - 2 * D[mask]

    return np.sum(diff**2)

from sklearn.feature_extraction.text import TfidfVectorizer
def tf_idf_embeddings(texts):
    vectorizer = TfidfVectorizer(max_features=1000)
    tfidf_matrix = vectorizer.fit_transform(texts)
    return tfidf_matrix.toarray()


In [53]:
paper_data = json.load(open("HCI_paper_abstracts/papers.json"))
all_texts = [paper['content'] for paper in paper_data]
api_key = open("../api_key").read().strip()
# embeddings = await multithread_embeddings(all_texts, api_key)
embeddings = tf_idf_embeddings(all_texts)
embeddings = np.array(embeddings)
embeddings.shape
# for paper, embedding in zip(paper_data, embeddings):
#     paper['embedding'] = embedding
# save_json(paper_data, 'HCI_paper_abstracts/embeddings.json')


(1723, 1000)

In [37]:
paper_w_embeddings = json.load(open("HCI_paper_abstracts/embeddings.json"))
X = np.array([paper['embedding'] for paper in paper_w_embeddings])

(500, 1536)

In [55]:
sub_X = embeddings[:500]
angles = circular_dr(sub_X)
print(angles)

calculating pairwise distances
optimizing positions
[4.56809429 5.05492282 0.         4.59478144 4.50997288 3.82893479
 2.18112397 2.54690795 1.67421592 6.28318531 0.         6.14540623
 6.28318531 2.17731706 0.5060088  0.04009305 4.97195877 6.28318531
 0.         2.88038295 2.12074357 1.97749716 4.27047173 0.
 3.96875512 0.93582299 1.48283549 0.         4.13519709 3.65379142
 1.76577745 6.28318531 6.28318531 6.28318531 1.44226051 5.66518794
 3.87017157 6.28318531 4.24836739 2.20909904 3.39179363 3.20165325
 5.94499119 3.25100331 2.19394026 3.93238596 3.64091055 6.00668613
 2.35822724 0.18927305 1.82330915 4.86166641 0.         4.12237761
 6.1394515  3.29249681 4.76250306 4.73182337 4.64284566 6.15209357
 0.         2.44100292 4.24946237 4.77607356 1.17183635 0.12661623
 1.10474472 4.03468083 2.43532933 4.50645741 1.06255032 6.28318531
 1.65540786 0.9391001  6.28318531 4.96981155 2.53055702 3.89166601
 1.59462366 2.63412195 4.9358495  5.57117006 6.28318531 5.26913564
 1.24756148 2.3099